In [1]:
from nltk import word_tokenize, ngrams
import json
import csv
import pandas as pd
import numpy as np
import os
import re
import string

In [2]:
## Se carga e ¿l archivo de stop words
def stopwords (tmp_entrada):
    stop = open("C:/Users/deimer0214/PycharmProjects/tatacoa/stopwords.txt", "r",  encoding="utf8")
    lista_stop=[]
    for st in stop:
        lista_tmp=[]
        st=st[0:st[0].find('\n')]
        lista_stop.append(st)
    for tr in lista_stop:
        tmp_entrada = [x for x in tmp_entrada if x != tr]
    return tmp_entrada

In [3]:
## Se carga lematización para pasar todo a presente 
def CargarDiccionarioLemas():
    file = open("C:/Users/deimer0214/PycharmProjects/tatacoa/diccionarioLematizador.txt", "rb")
    lema_d={}
    for line in file:
        bloques = line.split()
        palabra = bloques[0]
        lema = bloques[1]
        lema_d.update({palabra:lema})
    return lema_d

In [4]:
## pdte
def lematizador(lema_d,palabra):
    palabra=palabra.lower()
    if palabra in lema_d:
        lema = str(lema_d.get(palabra))
    else:
        lema = palabra
    return lema

lema_d = CargarDiccionarioLemas()

In [74]:
## Carga de Json que sale de tweepy
with open('C:/Users/deimer0214/PycharmProjects/tatacoa/cien_tweets.json') as f:
    data = json.load(f)
    lista_registros = []
    for l in data["Name"]:
        tmpCampo=l['text']
        if (len(tmpCampo))>0:
            terminos = {}
            lineaSplit = word_tokenize(tmpCampo.lower())
            for palabra in lineaSplit:
                terminos[palabra] = 1
            tmp_salida = stopwords(terminos)
            lista_palabras = []
            for palabra in tmp_salida:
                lista_palabras.append(lematizador(lema_d, palabra))
            #lista_registros = {"Vector": lista_palabras}
            lista_registros.append(lista_palabras)

Aca en adelante procurar hacerlo bien

In [82]:
type(data)

dict

In [118]:
#data['Name'][42]['text']
df_in = pd.DataFrame(data['Name'])
data2=df_in[['timestamp_ms','text']]
data2.head()

,timestamp_ms,text
0,1567481491044,RT @ShefVaidya: Will all the Indian Americans ...
1,1567481492155,RT @TomFitton: Obama used Comey to spy on @Rea...
2,1567481493104,RT @JudicialWatch: Hillary Clinton is not abov...
3,1567481493810,"RT @dmills3710: Devlin Barrett, As Reporter, @..."
4,1567481495499,RT @charliekirk11: RT if\n\nJames Comey\n\nJoh...


# Text Cleaning

In [123]:
def clean_text(txt):
    txt="".join([c for c in txt if c not in string.puntuation])
    tokens = re.split('\w+', txt)
    txt=[ps.stem(word) for word in tokens if wordnot in stopwords]
    return txt

# TFIDF Vectorizer

In [140]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()

corpus = ["This is a sentence is",
          "This is another sentence",
          "Third document is here"]

X = tfidf_vect.fit(corpus)
print(X.vocabulary_)
print(tfidf_vect.get_feature_names())

X = tfidf_vect.transform(corpus)
print(X.shape)
print(X)
print(X.toarray())

df = pd.DataFrame(X.toarray(), columns = tfidf_vect.get_feature_names())
print(df)

{'this': 6, 'is': 3, 'sentence': 4, 'another': 0, 'third': 5, 'document': 1, 'here': 2}
['another', 'document', 'here', 'is', 'sentence', 'third', 'this']
(3, 7)
  (0, 6)	0.4760629392767929
  (0, 4)	0.4760629392767929
  (0, 3)	0.7394106813498714
  (1, 6)	0.4804583972923858
  (1, 4)	0.4804583972923858
  (1, 3)	0.3731188059313277
  (1, 0)	0.6317450542765208
  (2, 5)	0.546454011634009
  (2, 3)	0.3227445421804912
  (2, 2)	0.546454011634009
  (2, 1)	0.546454011634009
[[0.         0.         0.         0.73941068 0.47606294 0.
  0.47606294]
 [0.63174505 0.         0.         0.37311881 0.4804584  0.
  0.4804584 ]
 [0.         0.54645401 0.54645401 0.32274454 0.         0.54645401
  0.        ]]
    another  document      here        is  sentence     third      this
0  0.000000  0.000000  0.000000  0.739411  0.476063  0.000000  0.476063
1  0.631745  0.000000  0.000000  0.373119  0.480458  0.000000  0.480458
2  0.000000  0.546454  0.546454  0.322745  0.000000  0.546454  0.000000
